In [ ]:
import numpy as np
import math
import copy

class details:
  def __init__(self,current_grid,heu_val = 0):
    self.current_grid =current_grid
    self.heu_val = heu_val
    self.g_n = 0
    self.list_of_child = []
    self.par_link = None

temp_grid = [3,2,1,4,5,6,8,7,0];
temp_grid[temp_grid.index(0)] = 'B'
start_matrix = [temp_grid[0:3],temp_grid[3:6],temp_grid[6:]]
input_grid = start_matrix

T = [[1,2,3],[4,5,6],[7,8,'B']]
target_grid = copy.deepcopy(T)
target_grid = [*target_grid[0],*target_grid[1],*target_grid[2]]
target_grid[target_grid.index('B')] = 0
target_grid = np.array(target_grid).reshape(3,3)



# Using Inversions To Check Whether The Grid Is Solvable or Not?
# The idea is No. of Inversions should be even else, Grid cannot be solved
# Inversion: if (i)th index element is strictly greater than the (j)th index where i<j (Going from Left to Right)
def is_grid_valid(input_grid):
  count_inversion = 0;
  grid_length = 9;
  if input_grid[-1] == 'B':
    grid_length = 8;
    for i in range(7):
      for j in range(i+1,8):
        if input_grid[i] == 'B' or input_grid[j] == 'B':
          continue;
        if (int(input_grid[i]) > int(input_grid[j])):
          count_inversion = count_inversion+1;
  return ((count_inversion%2) == 0);



def calculate_heuristics(child_list,curr_heu,heu_val = 0):
  if curr_heu == 'uninf_search':
    return heu_val
  elif curr_heu == 'misplaced_tiles':
    temp_children = [*child_list[0],*child_list[1],*child_list[2]]
    temp_target = target_grid.reshape(1,9)[0]
    for i in range(len(temp_children)):
      if(temp_children[i]!= 'B'):
        if(temp_children[i]!=temp_target[i]):
          heu_val+=1
    return heu_val
  elif curr_heu == 'manhattan_distance':
    temp_children = [*child_list[0],*child_list[1],*child_list[2]]
    temp_children[temp_children.index('B')] = 0
    temp_children=np.array(temp_children)
    for i in temp_children.reshape(1,9)[0]:
      if(i!=0):
        child_location = np.array(np.where(temp_children==i))
        target_location = np.array(np.where(target_grid == i))
        heu_val+=np.sum(np.absolute(child_location-target_location))
    return heu_val
  elif curr_heu == 'overestimate':
    temp_children = [*child_list[0],*child_list[1],*child_list[2]]
    temp_children[temp_children.index('B')] = 0
    temp_children=np.array(temp_children)
    for i in temp_children.reshape(1,9)[0]:
      if(i!=0):
        child_location = np.array(np.where(temp_children==i))
        target_location = np.array(np.where(target_grid==i))
        heu_val+=math.sqrt(np.sum((child_location-target_location)**2))
    return heu_val

In [ ]:
dictionary={}   
def get_children(current_state_format):
  current_grid = current_state_format.current_grid
  for i,j in enumerate(current_grid):
    if 'B' in j:
      bi = j.index('B')
      break

  next_coord = list(filter(lambda x:not((x[0]>2 or x[0]<0) or (x[1]>2 or x[1]<0)),[[i-1,bi],[i+1,bi],[i,bi-1],[i,bi+1]]))
  new_grids = []
  for m, n in next_coord:
    children = copy.deepcopy(current_grid)
    children[i][bi] = children[m][n]
    children[m][n] = 'B'
    new_grids.append(children)
  
  children_list= []

  for i in new_grids:
    tg = tuple(map(tuple,i))
    b_val = dictionary.get(tg,True)
    if(b_val):
      dictionary[tg] = False
      children_list.append(i)

  return children_list 


In [ ]:
import heapdict
import time
final_grid_format = []

def solver_method(heu_type):
    priority_queue = heapdict.heapdict()
    start_exe_time = time.time()
    grid_counter = 0
    node = details(input_grid,calculate_heuristics(input_grid,heu_type))
    node.g_n = 0
    priority_queue[node]=node.heu_val+node.g_n
    while True:
        parent_node,f_n = priority_queue.popitem()
        children_list = get_children(parent_node)
        grid_counter+=1
        if parent_node.current_grid == T:
            final_grid_format.append(copy.deepcopy(parent_node))
            print("target found")
            break
        for each_child in children_list:
            node = details(each_child,calculate_heuristics(each_child,heu_type))
            # parent_node.list_of_child.append(node)
            node.par_link = parent_node
            node.g_n = (node.par_link.g_n)+1
            priority_queue[node] = node.heu_val + node.g_n
    return time.time()-start_exe_time,len(dictionary.keys()),parent_node.g_n

In [ ]:
import pandas as pd
def setter_method():
  heuristic_types = ['uninf_search','misplaced_tiles','manhattan_distance','overestimate']
  param = ['Execution Time','Number of States','Number of States to Optimal Path']
  df = pd.DataFrame(0,index=param,columns=heuristic_types)
  for i in heuristic_types:
    for index,values in enumerate(solver_method(i)):
      df.loc[param[index],i] = values
    dictionary.clear()
    print("Execution Successful\n")
    print(f'{i} Heuristic Executed Successfully\n')
    print(df[i].round(2),'\n\n')

  print(df.round(2))
  return df

In [ ]:
def driver_method():
  print("\t\tINPUT GIVEN: ");
  ctr = 0;
  for i in input_grid:
    if(ctr%3 == 0):
      print("")

    print(i," ",end=" ");
    ctr= ctr+1;

  print("\n\t\tTARGET STATE: ");
  ctr = 0;
  for i in target_grid:
    if(ctr%3 == 0):
      print("")

    print(i," ",end=" ");
    ctr= ctr+1;

  df = setter_method()
  print(df)
  return

In [ ]:
driver_method()

		INPUT GIVEN: 

[3, 2, 1]   [4, 5, 6]   [8, 7, 'B']   
		TARGET STATE: 

[1 2 3]   [4 5 6]   [7 8 0]   target found
Execution Successful

uninf_search Heuristic Executed Successfully

Execution Time                          13.06
Number of States                    151020.00
Number of States to Optimal Path        24.00
Name: uninf_search, dtype: float64 


target found
Execution Successful

misplaced_tiles Heuristic Executed Successfully

Execution Time                          1.25
Number of States                    21553.00
Number of States to Optimal Path       24.00
Name: misplaced_tiles, dtype: float64 


target found
Execution Successful

manhattan_distance Heuristic Executed Successfully

Execution Time                         10.45
Number of States                    41165.00
Number of States to Optimal Path       24.00
Name: manhattan_distance, dtype: float64 


target found
Execution Successful

overestimate Heuristic Executed Successfully

Execution Time                  

In [ ]:
for i in final_grid_format:
  print(i.current_grid)
  print(i.heu_val)

[[1, 2, 3], [4, 5, 6], [7, 8, 'B']]
0
[]
[[1, 2, 3], [4, 5, 6], [7, 8, 'B']]
0
[]
[[1, 2, 3], [4, 5, 6], [7, 8, 'B']]
42
[]
[[1, 2, 3], [4, 5, 6], [7, 8, 'B']]
30.86954418941791
[]


In [ ]:
list_of_metrics = []
columns = ['uninf_search','misplaced_tiles','manhattan_distance','overestimate']
df = pd.DataFrame(index=[0,1,2,3],columns=columns)

for ind,i in enumerate(columns):
  depth = 0
  temp = final_grid_format[ind]
  while temp:
    list_of_metrics.append(temp.current_grid)
    temp = temp.par_link
    